In [1]:
import networkx as nx
import math
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import det
from numpy import cov

In [2]:
data = sio.loadmat('Market.mat')['DI']
data.shape

(7, 190, 12)

In [3]:
T = 7
P = 190
N = 12

In [4]:
#the nodes are the companies:
vertex_set = [i for i in range(12)]
G = nx.DiGraph()
for v in vertex_set:
    G.add_node(v)

In [5]:
all_nodes = set(vertex_set)

In [6]:
I = np.zeros((N,N))

In [7]:
for i in vertex_set:
    for j in vertex_set:
        if i == j:
            continue
        else:
            s_x = {i}
            s_y = {j}
            s_z = all_nodes - s_x - s_y
            x_t = data[:,:,i]  #company i
            y_t = data[:,:,j]  #company j

            z_t = [] #remaining companies
            for k in s_z:
                z_t.append(data[:,:,k])
                
            z_t = np.asarray(z_t)
            
            k = 0
            for t in np.arange(1,T+1):
                m1 = np.concatenate( (y_t[:t+1,:].reshape(-1,P),z_t[:,:t,:].reshape(-1,P)))
                m2 = np.concatenate( (x_t[:t,:].reshape(-1,P) , np.concatenate((y_t[:t,:].reshape(-1,P),z_t[:,:t,:].reshape(-1,P)))) )
                m3 = np.concatenate( (y_t[:t,:].reshape(-1,P),z_t[:,:t,:].reshape(-1,P)))
                m4 = np.concatenate( (x_t[:t,:].reshape(-1,P) , np.concatenate((y_t[:t+1,:].reshape(-1,P),z_t[:,:t,:].reshape(-1,P)))) )
        
                c1 = cov(m1)
                d1 = det(c1)

                c2 = cov(m2)
                d2 = det(c2)

                c3 = cov(m3)
                d3 = det(c3)

                c4 = cov(m4)                
                d4 = det(c4)
                
                print(d3*d4)
                k += np.log( (d1 * d2) / (d3 * d4) )
                
            DI = 0.5 * k
            I[i,j] = DI

3.031966461010392e-54
2.837088795853228e-112
4.432301329171123e-174
5.412551516152523e-239
1.570364466044017e-307
0.0


ZeroDivisionError: float division by zero

In [ ]:
I

In [ ]:
for i in vertex_set:
    for j in vertex_set:
        G.add_edge(i, j, weight = I[i,j])
nx.draw(G, with_labels = True)